# Sentiment Analysis using IndoBERT
This notebook focuses on finetuning IndoBERT on a specific dataset to do sentiment analysis task. This notebook runs on Google Colab using T4 GPU.

## Install dependencies and import libraries

In [2]:
# !pip install transformers datasets

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import random
import numpy as np
import pandas as pd
import torch
from torch import optim
import torch.nn.functional as F
from tqdm import tqdm

from transformers import BertForSequenceClassification, BertConfig, BertTokenizer

from utils.forward_fn import forward_sequence_classification
from utils.metrics import document_sentiment_metrics_fn
from utils.data_utils import DocumentSentimentDataset, DocumentSentimentDataLoader

In [5]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

def count_param(module, trainable=False):
    if trainable:
        return sum(p.numel() for p in module.parameters() if p.requires_grad)
    else:
        return sum(p.numel() for p in module.parameters())

def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']

def metrics_to_string(metric_dict):
    string_list = []
    for key, value in metric_dict.items():
        string_list.append('{}:{:.2f}'.format(key, value))
    return ' '.join(string_list)

In [6]:
# Set random seed
# So that the finetuning process can be remade
set_seed(2023)

In [7]:
# Define device
import torch
if torch.cuda.is_available():
  device = torch.device("cuda")
else:
  device = torch.device("cpu")

## Import model and tokenizer

In [ ]:
# Load Tokenizer and Config
tokenizer = BertTokenizer.from_pretrained('indobenchmark/indobert-base-p1')
config = BertConfig.from_pretrained('indobenchmark/indobert-base-p1')

# Instantiate model
model = BertForSequenceClassification.from_pretrained(
    'indobenchmark/indobert-base-p1',
    num_labels = 3,
    output_attentions = False,
    output_hidden_states = False
).to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at indobenchmark/indobert-base-p1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Import dataset

In [8]:
train_dataset_path = "/content/train_preprocess.tsv"
valid_dataset_path = "/content/valid_preprocess.tsv"
test_dataset_path = "/content/test_preprocess_masked_label.tsv"

In [9]:
train_dataset = DocumentSentimentDataset(train_dataset_path, tokenizer, lowercase=True)
valid_dataset = DocumentSentimentDataset(valid_dataset_path, tokenizer, lowercase=True)
test_dataset = DocumentSentimentDataset(test_dataset_path, tokenizer, lowercase=True)

train_loader = DocumentSentimentDataLoader(dataset=train_dataset, max_seq_len=512, batch_size=8, num_workers=2, shuffle=True)
valid_loader = DocumentSentimentDataLoader(dataset=valid_dataset, max_seq_len=512, batch_size=8, num_workers=2, shuffle=False)
test_loader = DocumentSentimentDataLoader(dataset=test_dataset, max_seq_len=512, batch_size=8, num_workers=2, shuffle=False)

NameError: ignored

In [10]:
w2i, i2w = DocumentSentimentDataset.LABEL2INDEX, DocumentSentimentDataset.INDEX2LABEL
print(w2i)
print(i2w)

{'positive': 0, 'neutral': 1, 'negative': 2}
{0: 'positive', 1: 'neutral', 2: 'negative'}


## Inference before finetuning

In [ ]:
text = 'Bahagia hatiku melihat pernikahan putri sulungku yang cantik jelita'
subwords = tokenizer.encode(text)
subwords = torch.LongTensor(subwords).view(1, -1).to(model.device)

logits = model(subwords)[0]
label = torch.topk(logits, k=1, dim=-1)[1].squeeze().item()

print(f'Text: {text} | Label : {i2w[label]} ({F.softmax(logits, dim=-1).squeeze()[label] * 100:.3f}%)')

Text: Bahagia hatiku melihat pernikahan putri sulungku yang cantik jelita | Label : neutral (40.933%)


It can be seen that the result is not as expected where the sentiment should be positive but the model recognize it as negative.

## Training
We finetune the model with learning rate 0.00003 in 10 epochs.

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=3e-6)
model = model.cuda()

In [ ]:
import time

# Train
n_epochs = 10
start = time.time()
for epoch in range(n_epochs):
    model.train()
    torch.set_grad_enabled(True)

    total_train_loss = 0
    list_hyp, list_label = [], []

    train_pbar = tqdm(train_loader, leave=True, total=len(train_loader))
    for i, batch_data in enumerate(train_pbar):
        # Forward model
        loss, batch_hyp, batch_label = forward_sequence_classification(model, batch_data[:-1], i2w=i2w, device='cuda')

        # Update model
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        tr_loss = loss.item()
        total_train_loss = total_train_loss + tr_loss

        # Calculate metrics
        list_hyp += batch_hyp
        list_label += batch_label

        train_pbar.set_description("(Epoch {}) TRAIN LOSS:{:.4f} LR:{:.8f}".format((epoch+1),
            total_train_loss/(i+1), get_lr(optimizer)))

    # Calculate train metric
    metrics = document_sentiment_metrics_fn(list_hyp, list_label)
    print("(Epoch {}) TRAIN LOSS:{:.4f} {} LR:{:.8f}".format((epoch+1),
        total_train_loss/(i+1), metrics_to_string(metrics), get_lr(optimizer)))

    # Evaluate on validation
    model.eval()
    torch.set_grad_enabled(False)

    total_loss, total_correct, total_labels = 0, 0, 0
    list_hyp, list_label = [], []

    pbar = tqdm(valid_loader, leave=True, total=len(valid_loader))
    for i, batch_data in enumerate(pbar):
        batch_seq = batch_data[-1]
        loss, batch_hyp, batch_label = forward_sequence_classification(model, batch_data[:-1], i2w=i2w, device='cuda')

        # Calculate total loss
        valid_loss = loss.item()
        total_loss = total_loss + valid_loss

        # Calculate evaluation metrics
        list_hyp += batch_hyp
        list_label += batch_label
        metrics = document_sentiment_metrics_fn(list_hyp, list_label)

        pbar.set_description("VALID LOSS:{:.4f} {}".format(total_loss/(i+1), metrics_to_string(metrics)))

    metrics = document_sentiment_metrics_fn(list_hyp, list_label)
    print("(Epoch {}) VALID LOSS:{:.4f} {}".format((epoch+1),
        total_loss/(i+1), metrics_to_string(metrics)))

stop = time.time()
print(f"\n\nTraining time: {stop - start}s")

(Epoch 1) TRAIN LOSS:0.2558 LR:0.00000300: 100%|██████████| 1375/1375 [03:13<00:00,  7.12it/s]


(Epoch 1) TRAIN LOSS:0.2558 ACC:0.91 F1:0.87 REC:0.86 PRE:0.89 LR:0.00000300


VALID LOSS:0.1801 ACC:0.93 F1:0.91 REC:0.90 PRE:0.92: 100%|██████████| 158/158 [00:08<00:00, 18.08it/s]


(Epoch 1) VALID LOSS:0.1801 ACC:0.93 F1:0.91 REC:0.90 PRE:0.92


(Epoch 2) TRAIN LOSS:0.1306 LR:0.00000300: 100%|██████████| 1375/1375 [03:14<00:00,  7.07it/s]


...

(Epoch 9) VALID LOSS:0.3065 ACC:0.93 F1:0.90 REC:0.89 PRE:0.92


(Epoch 10) TRAIN LOSS:0.0098 LR:0.00000300: 100%|██████████| 1375/1375 [03:15<00:00,  7.05it/s]


(Epoch 10) TRAIN LOSS:0.0098 ACC:1.00 F1:1.00 REC:1.00 PRE:1.00 LR:0.00000300


VALID LOSS:0.3076 ACC:0.93 F1:0.91 REC:0.90 PRE:0.91: 100%|██████████| 158/158 [00:08<00:00, 17.66it/s]

(Epoch 10) VALID LOSS:0.3076 ACC:0.93 F1:0.91 REC:0.90 PRE:0.91


Training time: 2037.5092387199402s


In [ ]:
model.eval()
torch.set_grad_enabled(False)

total_loss, total_correct, total_labels = 0, 0, 0
list_hyp, list_label = [], []

pbar = tqdm(test_loader, leave=True, total=len(test_loader))
for i, batch_data in enumerate(pbar):
    _, batch_hyp, _ = forward_sequence_classification(model, batch_data[:-1], i2w=i2w, device='cuda')
    list_hyp += batch_hyp

# Save prediction
df = pd.DataFrame({'label':list_hyp}).reset_index()
df.to_csv('pred.txt', index=False)

df.head()

100%|██████████| 63/63 [00:01<00:00, 39.13it/s]


,index,label
0,0,negative
1,1,negative
2,2,negative
3,3,negative
4,4,negative


## Inference after finetuning

In [ ]:
text = 'Bahagia hatiku melihat pernikahan putri sulungku yang cantik jelita'
subwords = tokenizer.encode(text)
subwords = torch.LongTensor(subwords).view(1, -1).to(model.device)

logits = model(subwords)[0]
label = torch.topk(logits, k=1, dim=-1)[1].squeeze().item()

print(f'Text: {text} | Label : {i2w[label]} ({F.softmax(logits, dim=-1).squeeze()[label] * 100:.3f}%)')

Text: Bahagia hatiku melihat pernikahan putri sulungku yang cantik jelita | Label : positive (99.980%)


After being finetuned, the model is able to classify the sentiment correctly.

## Save the model
Save the model and try to load it one more time to ensure that the model works fine.

In [ ]:
model.save_pretrained("/content")
tokenizer.save_pretrained("/content")

('/content/tokenizer_config.json',
 '/content/special_tokens_map.json',
 '/content/vocab.txt',
 '/content/added_tokens.json')

In [ ]:
saved_model = BertForSequenceClassification.from_pretrained("/content")
saved_tokenizer = BertTokenizer.from_pretrained("/content")

text = 'Bahagia hatiku melihat pernikahan putri sulungku yang cantik jelita'
subwords = saved_tokenizer.encode(text)
subwords = torch.LongTensor(subwords).view(1, -1).to(saved_model.device)

logits = saved_model(subwords)[0]
label = torch.topk(logits, k=1, dim=-1)[1].squeeze().item()

print(f'Text: {text} | Label : {i2w[label]} ({F.softmax(logits, dim=-1).squeeze()[label] * 100:.3f}%)')

Text: Bahagia hatiku melihat pernikahan putri sulungku yang cantik jelita | Label : positive (99.980%)


## Performance

In [11]:
test_dataset_path = "/content/test_preprocess.tsv"
df_test = pd.read_table(test_dataset_path, header=None)
df_test.rename(columns={0: "text", 1: "label"}, inplace=True)
df_test.head()

,text,label
0,kemarin gue datang ke tempat makan baru yang a...,negative
1,kayak nya sih gue tidak akan mau balik lagi ke...,negative
2,"kalau dipikir-pikir , sebenarnya tidak ada yan...",negative
3,ini pertama kalinya gua ke bank buat ngurusin ...,negative
4,waktu sampai dengan gue pernah disuruh ibu lat...,negative


In [13]:
# Load Tokenizer and Config
tokenizer = BertTokenizer.from_pretrained('indobenchmark/indobert-base-p1')
config = BertConfig.from_pretrained('indobenchmark/indobert-base-p1')

# Instantiate model
model = BertForSequenceClassification.from_pretrained(
    'indobenchmark/indobert-base-p1',
    num_labels = 3,
    output_attentions = False,
    output_hidden_states = False
).to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at indobenchmark/indobert-base-p1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
def infer(text):
  print(text)
  inputs = tokenizer.encode(text)
  inputs = torch.LongTensor(inputs).view(1, -1).to(model.device)

  logits = model(inputs)[0]
  label = torch.topk(logits, k=1, dim=-1)[1].squeeze().item()
  return i2w[label]

In [15]:
df_test['ori_pred'] = df_test['text'].apply(infer)
df_test.head()

kemarin gue datang ke tempat makan baru yang ada di dago atas . gue kira makanan nya enak karena harga nya mahal . ternyata , boro-boro . tidak mau lagi deh ke tempat itu . sudah mana tempat nya juga tidak nyaman banget , terlalu sempit .
kayak nya sih gue tidak akan mau balik lagi ke tempat itu . gila , ya , gue enggak ngerti kenapa tempat nya dibiarkan panas . sudah begitu kotor pula . kalau panas kepanasan , kalau hujan kehujanan . harus nya sih tidak ada restoran yang kayak gitu . tidak tahu deh apa yang mereka jual .
kalau dipikir-pikir , sebenarnya tidak ada yang bisa dibanggakan dari jokowi . pertama , dia tidak bisa nepatin janji . kedua , kerjaan nya selalu pencitraan . ketiga , dia tidak pro rakyat . sudahlah . ku sudah terlanjur kecewa .
ini pertama kalinya gua ke bank buat ngurusin pembuatan rekening baru . nama nya juga orang pertama kali ya baru ke bank , gua kena semprot . kelihatan banget pelayanan pelanggan - nya tidak suka gua banyak bertanya . amit-amit . padahal itu

,text,label,ori_pred
0,kemarin gue datang ke tempat makan baru yang a...,negative,positive
1,kayak nya sih gue tidak akan mau balik lagi ke...,negative,negative
2,"kalau dipikir-pikir , sebenarnya tidak ada yan...",negative,negative
3,ini pertama kalinya gua ke bank buat ngurusin ...,negative,positive
4,waktu sampai dengan gue pernah disuruh ibu lat...,negative,negative


In [16]:
ft_model = BertForSequenceClassification.from_pretrained("/content/drive/MyDrive/Models/indobert/")
ft_tokenizer = BertTokenizer.from_pretrained("/content/drive/MyDrive/Models/indobert/")

In [17]:
def infer_ft(text):
  print(text)
  inputs = ft_tokenizer.encode(text)
  inputs = torch.LongTensor(inputs).view(1, -1).to(ft_model.device)

  logits = ft_model(inputs)[0]
  label = torch.topk(logits, k=1, dim=-1)[1].squeeze().item()
  return i2w[label]

In [18]:
df_test['ft_pred'] = df_test['text'].apply(infer_ft)
df_test.head()

kemarin gue datang ke tempat makan baru yang ada di dago atas . gue kira makanan nya enak karena harga nya mahal . ternyata , boro-boro . tidak mau lagi deh ke tempat itu . sudah mana tempat nya juga tidak nyaman banget , terlalu sempit .
kayak nya sih gue tidak akan mau balik lagi ke tempat itu . gila , ya , gue enggak ngerti kenapa tempat nya dibiarkan panas . sudah begitu kotor pula . kalau panas kepanasan , kalau hujan kehujanan . harus nya sih tidak ada restoran yang kayak gitu . tidak tahu deh apa yang mereka jual .
kalau dipikir-pikir , sebenarnya tidak ada yang bisa dibanggakan dari jokowi . pertama , dia tidak bisa nepatin janji . kedua , kerjaan nya selalu pencitraan . ketiga , dia tidak pro rakyat . sudahlah . ku sudah terlanjur kecewa .
ini pertama kalinya gua ke bank buat ngurusin pembuatan rekening baru . nama nya juga orang pertama kali ya baru ke bank , gua kena semprot . kelihatan banget pelayanan pelanggan - nya tidak suka gua banyak bertanya . amit-amit . padahal itu

,text,label,ori_pred,ft_pred
0,kemarin gue datang ke tempat makan baru yang a...,negative,positive,negative
1,kayak nya sih gue tidak akan mau balik lagi ke...,negative,negative,negative
2,"kalau dipikir-pikir , sebenarnya tidak ada yan...",negative,negative,negative
3,ini pertama kalinya gua ke bank buat ngurusin ...,negative,positive,negative
4,waktu sampai dengan gue pernah disuruh ibu lat...,negative,negative,negative


In [19]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

d = {
    "Accuracy": [accuracy_score(df_test['label'], df_test['ori_pred']),
                 accuracy_score(df_test['label'], df_test['ft_pred']),],
    "Precision":[precision_score(df_test['label'], df_test['ori_pred'], average="macro"),
                 precision_score(df_test['label'], df_test['ft_pred'], average="macro")],
    "Recall":   [recall_score(df_test['label'], df_test['ori_pred'], average="macro"),
                 recall_score(df_test['label'], df_test['ft_pred'], average="macro")],
    "F1":       [f1_score(df_test['label'], df_test['ori_pred'], average="macro"),
                 f1_score(df_test['label'], df_test['ft_pred'], average="macro")]
}

df_comp = pd.DataFrame.from_dict(d)
df_comp = df_comp.rename(index={0: 'Original BERT', 1: 'Finetuned BERT'})
df_comp.to_csv('indobert_performance.csv', index=False)
df_comp

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,Accuracy,Precision,Recall,F1
Original BERT,0.416,0.266154,0.339147,0.248810
Finetuned BERT,0.916,0.915580,0.875811,0.890512


The results above shows that finetuning process is important and necessary before applying the model to a specific task. In this case, finetuning show that it could improve the model's performance significantly (± 0.5 for accuracy and ± 0.65 for f1-score).